In [ ]:
!pip install regex
!pip install requests_html

In [ ]:
# Packages config
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
# Variaveis de ambiente
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
 
# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [ ]:
import pandas as pd
from pyspark.sql import *
from pyspark.sql.types import *

In [ ]:
# WEB SCRAPING LINK ARQUIVO
from requests_html import HTMLSession
from requests_html import AsyncHTMLSession

session = HTMLSession() # Initialize the GET requests
url = 'https://opendatasus.saude.gov.br/ro/dataset/bd-srag-2020'
r = session.get(url) 

descriptions_wheather_of_forecast =  r.html.find('.resource-url-analytics')

link = descriptions_wheather_of_forecast[2].attrs['href']
name_file = link[-21:]

In [ ]:
# https://www.cetax.com.br/blog/tutorial-pyspark-e-mllib/

In [ ]:
# PYSPARK
from pyspark import SparkFiles
spark = SparkSession.builder.master('local[*]').getOrCreate()
spark.sparkContext.addFile(link)
df = spark.read.csv("file://"+SparkFiles.get(name_file),sep=';', header=True, inferSchema= True)

df.printSchema()

root
 |-- DT_NOTIFIC: string (nullable = true)
 |-- SEM_NOT: integer (nullable = true)
 |-- DT_SIN_PRI: string (nullable = true)
 |-- SEM_PRI: integer (nullable = true)
 |-- SG_UF_NOT: string (nullable = true)
 |-- ID_REGIONA: string (nullable = true)
 |-- CO_REGIONA: integer (nullable = true)
 |-- ID_MUNICIP: string (nullable = true)
 |-- CO_MUN_NOT: integer (nullable = true)
 |-- ID_UNIDADE: string (nullable = true)
 |-- CO_UNI_NOT: integer (nullable = true)
 |-- CS_SEXO: string (nullable = true)
 |-- DT_NASC: string (nullable = true)
 |-- NU_IDADE_N: integer (nullable = true)
 |-- TP_IDADE: integer (nullable = true)
 |-- COD_IDADE: string (nullable = true)
 |-- CS_GESTANT: integer (nullable = true)
 |-- CS_RACA: integer (nullable = true)
 |-- CS_ETINIA: string (nullable = true)
 |-- CS_ESCOL_N: integer (nullable = true)
 |-- ID_PAIS: string (nullable = true)
 |-- CO_PAIS: integer (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- ID_RG_RESI: string (nullable = true)
 |-- CO

In [ ]:
df.show()

In [ ]:
df.createOrReplaceTempView("table1")

query = """ 

    SELECT DISTINCT CS_SEXO 
    FROM table1 AS TB1
    WHERE TB1.CS_SEXO = 'M'

"""
df2 = spark.sql(query)

In [ ]:
df2.show(5)